# SDC Polyglot Workbook 

This work book is an example of what can be done with SDC in Polyglot notebook. 

In [ ]:
 <Section name="S_4257" order="19" ID="4257.100004300" minCard="0">
 </Section>

## Sections
 
The code snippet above shows a section with a name, order, ID and minCard attribute

# Questions

Each QAS in the FDF XML is an XFC that can be represented in the DEF by one or more controls (widgets) by the FF software. The controls capture user response data in the DEF.  Each QAS XFC has an ID   and other metadata. DEF controls are assigned the matching XFC ID to enable data from the DEF control to be captured into the XFC XML that shares its @ID. The behavior of the control is determined by the corresponding XFC metadata. There are 3 QAS types: Question-Response, Single-Select Question, and Multi-Select Question.

The primary visible text for all controls is found in the title attribute inside the `<Question>` element.  

## Question-Response
**Observe the following points about this sample:**

- *Question* is the only XFC that appears in this XML example. The other nested elements support the *Question XFC* with additional metadata.

- All XFC elements (*Question* in this case) must have an **ID** that is unique within the FDF. This uniqueness is enforced by the XML Schema. Uniqueness of IDs must be confirmed by SDC-compliant software. Since XFCs can be repeated in DEF implementation, uniqueness enforcement is an important task in SDC software implementations.

- The text to display in the DEF is in the *title content*.

- **Responses** are captured in a `ResponseField / Response / (datatype)` structure, where `(datatype)` is an SDC element corresponding to one of the SDC datatypes.

  - Responses must contain an SDC datatype element (*string* in this case).

  - User-entered responses are captured in the `val` attribute of the `(datatype)` element, like this:
    ```xml
    <string maxLength="4000" minLength="0" val="This is my comment" />
    ```

    The example below offers a complete example of a `<Question>` element. 

In [ ]:
<Question ID="40273.100004300" title="Comment(s)">
    <ResponseField>
        <Response>
            <string maxLength="4000" minLength="0"/>
        </Response>
    </ResponseField>
</Question>

In [ ]:
<Question ID="QM1" title="This is a QM Question title" >
    <ListField maxSelections="0">
        <List>
            <ListItem ID="LI1" title="ListItem.title.1" />
            <ListItem ID="LI2" title="ListItem.title.2" />
            <ListItem ID="LI3" title="ListItem.title.3" />
        </List>
    </ListField>
</Question>

## minCard

@minCard is the minimum number of repetitions allowed for a section or question. The default value is 1, indicating that the user must answer any question(s) that has this attribute set. If @minCard is set to 0, then the item and all descendent questions are optional to answer.


In [ ]:
<Question name="Q_2118" order="21" ID="2118.100004300" minCard="0" readOnly="true">
      <Property name="p_altTxt_2118_1" order="22" propName="altText" val="Tumor Site" />
      <ListField name="lf_2118_2" order="23">
       <List name="lst_2118_3" order="24">
        <ListItem name="LI_2119" order="25" ID="2119.100004300" title="Adrenal gland" selected="true">
         <Property name="p_rptTxt_2119_1" order="26" propName="reportText" val="{no text}" />
        </ListItem>
       </List>
      </ListField>
     </Question>

## Read Only Question

The question above is a special kind of read only question that is displayed to the user, but they cannot fill it in. It is informational in nature only. 

In [ ]:
 <ListItem name="LI_45594" order="192" ID="45594.100004300" title="Other adjacent organs and structures (specify)">
         <Property name="p_rptTxt_45594_1" order="193" propName="reportText" val="{no text}" />
         <ListItemResponseField name="lirf_45594_2" order="194" responseRequired="true">
          <Response name="rsp_45594_3" order="195">
           <string name="str_45594_4" order="196" />
          </Response>
         </ListItemResponseField>
        </ListItem>

## ListItem With Response

There are many questions in SDC forms which might have a response section 